# Exploring Playa Inundation & Area Over Time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
inun_csv_basename = '../data/state_county_csvs/inun_frac_'
state_list = ['CO', 'KS', 'NE', 'NM', 'OK','TX']

In [ ]:
# Hardcoded number of months and years in dataset
num_months = 418
num_years = 35

## Counting number of 0s per state, and saving csvs

In [ ]:
state_zeros_df = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path)
    zeros = (state_df[['id','inundation']].groupby('id').max()==0).sum()
    nonzeros = state_df['id'].unique().shape[0] - zeros
    state_zeros_df = state_zeros_df.append(pd.DataFrame(
        {'state':state,'zeros':zeros, 'nonzeros':nonzeros}))
    
state_zeros_df = state_zeros_df.set_index('state')
state_zeros_df = state_zeros_df.assign(
    total=state_zeros_df['zeros'] + state_zeros_df['nonzeros'])
state_zeros_df = state_zeros_df.assign(
    frac_zeros=state_zeros_df['zeros']/state_zeros_df['total'])

In [ ]:
print(state_zeros_df)

# Zeros vs non-zeros by source

In [ ]:
playa_att_csv_path = '../data/playa_nogeometry.csv'
playa_att_df = pd.read_csv(playa_att_csv_path).set_index('id')

In [ ]:
# Group by authors and sources
author_zeros_df = pd.DataFrame()
source_zeros_df = pd.DataFrame()
author_by_state = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path).set_index('id')
    state_df.index = state_df.index.set_names('id')
    playa_att_state_df = playa_att_df.loc[playa_att_df['state']==state,['author','source']]
    state_df = state_df[['inundation']].join(playa_att_state_df,how='inner')
    
    # Author stats
    author_id_grp = state_df[['inundation','author']].groupby(['author','id'])
    zeros = (author_id_grp.max()==0).groupby('author').sum()['inundation']
    total = playa_att_state_df.groupby('author').size()
    nonzeros = total-zeros
    state_author_zeros_df = pd.DataFrame({'zeros':zeros,
                                          'nonzeros':nonzeros,
                                          'total':total})
    author_zeros_df = author_zeros_df.append(state_author_zeros_df)
    
    # Source stats
    source_id_grp = state_df[['inundation','source']].groupby(['source','id'])
    zeros = (source_id_grp.max()==0).groupby('source').sum()['inundation']
    total = playa_att_state_df.groupby('source').size()
    nonzeros = total-zeros
    state_source_zeros_df = pd.DataFrame({'zeros':zeros,
                                          'nonzeros':nonzeros,
                                          'total':total})
    source_zeros_df = source_zeros_df.append(state_source_zeros_df)
    
    # State author stats
    state_authors = state_df[['author']].assign(id=state_df.index).groupby('author').agg({'id':'nunique'})
    state_authors = state_authors.assign(state=state)
    author_by_state = author_by_state.append(state_authors)

# Final grouping of authors
author_zeros_df = author_zeros_df.groupby('author').sum()
author_zeros_df = author_zeros_df.assign(
    zero_frac = author_zeros_df['zeros']/author_zeros_df['total']
)

# Final grouping of sources
source_zeros_df = source_zeros_df.groupby('source').sum()
source_zeros_df = source_zeros_df.assign(
    zero_frac = source_zeros_df['zeros']/source_zeros_df['total']
)

# Final grouping of authors
author_by_state = author_by_state.reset_index().pivot(index='state',columns='author',values='id')

In [ ]:
print(author_zeros_df,'\n\n')
print(source_zeros_df,'\n\n')
print(author_by_state)


## Get basic per-lake stats

In [ ]:
def months_nonzero(x):
    return (x>0).sum()

def years_nonzero(x):
    return max(x)>0

In [ ]:
def calc_per_playa_stats(state_df):

    # Might be way to combine into one groupby, but this works pretty fast
    per_playa = state_df[['id','inundation', 'area']].groupby('id').agg(['max','min','sum','mean','median',months_nonzero])
    years_nonzero_df = state_df[['id','inundation', 'year']].groupby(['id', 'year']).agg({'inundation':years_nonzero}).groupby('id').sum()
    per_playa = per_playa.assign(years_nonzero=years_nonzero_df['inundation'])
    
    return per_playa

In [ ]:
per_playa_df = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path)
    state_per_playa = calc_per_playa_stats(state_df)
    per_playa_df = per_playa_df.append(state_per_playa)

In [ ]:
print((per_playa_df['inundation']['max']==0).sum())
print((per_playa_df['inundation']['max']>0).sum())

In [ ]:
ax = per_playa_df.loc[per_playa_df['inundation']['max']>0]['inundation']['max'].hist(figsize=[12,8])
ax.set_xlabel('Inundation Fraction', size=18)
ax.set_ylabel('Count', size=18)
ax.tick_params(axis="x", labelsize=14)
ax.tick_params(axis="y", labelsize=14)
plt.show()

In [ ]:
# For playas that were inundated at some point, on avg how many months were they inundated?
per_playa_df.loc[per_playa_df['inundation']['max']>0,
                 per_playa_df.columns.get_level_values(1)=='months_nonzero']['inundation'].mean() # On average, 1/10 months

In [ ]:
# # For playas that were inundated at some point, on avg how many months were they inundated?
# per_playa_df.loc[per_playa_df['inundation']['max']>0,
#                  per_playa_df.columns.get_level_values(1)=='years_nonzero'].mean() # On average, 1/10 months

## Save per-playa stats as csv

In [ ]:
per_playa_df.assign(nonzero = per_playa_df['inundation']['max']>0)
per_playa_df.to_csv('../data/jrc-water_summary_atts.csv')

## Get basic per-year stats

In [ ]:
def calc_per_year_stats(state_df,state):

    per_year = state_df[['inundation','area','year']].groupby('year').agg(['max','min','sum','mean','median',months_nonzero])
    per_year = per_year.assign(state=state)
    return per_year

In [ ]:
per_year_df = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path)
    state_per_year= calc_per_year_stats(state_df, state)
    per_year_df = per_year_df.append(state_per_year)
per_year_df.index.rename('year',inplace=True)

In [ ]:
per_year_df['inundation'].groupby('year').agg({'sum':'sum', 'months_nonzero':'sum'}).plot()

In [ ]:
params = {'legend.fontsize': 14,
          'legend.handlelength': 2}
plt.rcParams.update(params)
ax = per_year_df['area'].groupby('year').agg({'max':'sum'}).plot(figsize=[12,6], lw=2, legend=False)
ax.set_title('Inundated Area (acres)', size=20)
ax.set_xlabel('Year', size=18)
ax.set_ylabel('Max Inundated Area', size=18)
ax.tick_params(axis="x", labelsize=14)
ax.tick_params(axis="y", labelsize=14)
plt.show()

## Basic per-month stats

In [ ]:
def calc_per_month_stats(state_df, state):

    per_month = state_df[['inundation','area','month']].groupby(['month']).agg(['max','min','sum','mean','median'])
    per_month = per_month.assign(state=state)
               
    return per_month

In [ ]:
per_month_df = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path)
    state_per_month= calc_per_month_stats(state_df, state)
    per_month_df = per_month_df.append(state_per_month)
per_month_df.index.rename('month',inplace=True)

In [ ]:
# per_month_df['inundation'].groupby('month').agg({'sum':'sum'}).plot(title="Sum of Inundation")

In [ ]:
per_month_df['area'].groupby('month').agg({'sum':'sum'}).plot(title="Monthly Inundated Area (Acres)")

In [ ]:
per_month_df.assign(sum_area = per_month_df['area']['sum'])[['state','sum_area']].pivot(
    index=None, columns='state', values='sum_area').plot(title='Monthly Inundated Area, by state')

## Get basic per-month/year stats

In [ ]:
def calc_per_month_year_stats(state_df, state):

    per_month_year = state_df[['inundation','area','year','month']].groupby(['year','month']).agg(['max','min','sum','mean','median'])
    per_month_year = per_month_year.assign(state=state)
               
    return per_month_year

In [ ]:
per_month_year_df = pd.DataFrame()
for state in state_list:
    state_csv_path = inun_csv_basename + state + '.csv'
    state_df = pd.read_csv(state_csv_path)
    state_per_month_year = calc_per_month_year_stats(state_df, state)
    per_month_year_df = per_month_year_df.append(state_per_month_year)
per_month_year_df.index.rename(['year','month'],inplace=True)

In [ ]:
# per_month_year_df['inundation'].groupby(['year','month']).agg({'sum':'sum'}).plot()

In [ ]:
per_month_year_df['area'].groupby(['year','month']).agg({'max':'sum'}).plot()